In [23]:
from cameo import models
from cameo import load_model
import pandas as pd
import numpy as np
from cobra import Model, Reaction, Metabolite
import cobra

In [2]:
def evaluate_knockout(model, mutant, ko_list):
    bound_knockout = []
    for i in ko_list:
        i_gene = str(model.genes[i])
        i_uniprot = str(model.genes[i].annotation['uniprot'][0])
        i_n_reactions = len(model.genes[i].reactions)
        i_reactions = []
        i_bounds_model = []
        i_bounds_mutant = []
        for x in model.genes[i].reactions:
            x_reaction = [str(x.id), str(x.reaction)]
            x_bounds = x.bounds
            i_reactions.append(x_reaction)
            i_bounds_model.append(x_bounds)
        for y in mutant.genes[i].reactions:
            y_bounds = y.bounds
            i_bounds_mutant.append(y_bounds)
        data = [i_gene, i_uniprot, i_n_reactions, i_reactions, i_bounds_model, i_bounds_mutant]
        bound_knockout.append(data)

    for i in bound_knockout:
        print('gene: '+i[0])
        print('uniprot: '+i[1])
        print('number of reactions involved: '+str(i[2]))
        for count, x in enumerate(i[3]):
            print('* '+x[0]+' :', x[1])
            print('bounds normal: '+str(i[4][count]))
            print('bounds mutant: '+str(i[5][count]))
        print('------------------------------------------------------------')
    return

In [3]:
model = load_model('iML1515')
model.solver = 'gurobi'
model

Using license file /home/matinnu/gurobi.lic
Academic license - for non-commercial use only


Name,iML1515
Memory address,0x07f202651c1d0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,0.0 + 1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Xylitol strain knockout model

In [4]:
# copy model to create mutant
mutant_xylitol = model.copy()

# create list of genes to KO
ko_list = pd.read_csv('data/df_xylitol_knockouts.csv', index_col=0).dropna(subset = ['id_bigg'])
ko_list.iML1515_idx = ko_list.iML1515_idx.astype(int)
#ko_list

Read LP format model from file /tmp/tmpovhx5czb.lp
Reading time = 0.03 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /tmp/tmpb9ak0l9u.lp
Reading time = 0.02 seconds
: 1877 rows, 5424 columns, 21150 nonzeros


In [5]:
for i in ko_list.iML1515_idx:
    mutant_xylitol.genes[i].knock_out()
    
#model = model
#mutant = mutant_xylitol
#ko_list = ko_list.iML1515_idx

evaluate_knockout(model, mutant_xylitol, ko_list.iML1515_idx)

gene: b0904
uniprot: P0AC23
number of reactions involved: 2
* FORtppi : for_c --> for_p
bounds normal: (0.0, 1000.0)
bounds mutant: (0.0, 1000.0)
* FORt2pp : for_p + h_p --> for_c + h_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0.0, 1000.0)
------------------------------------------------------------
gene: b0903
uniprot: P09373
number of reactions involved: 2
* PFL : coa_c + pyr_c --> accoa_c + for_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0.0, 1000.0)
* OBTFL : 2obut_c + coa_c --> for_c + ppcoa_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0.0, 1000.0)
------------------------------------------------------------
gene: b1380
uniprot: P52643
number of reactions involved: 1
* LDH_D : lac__D_c + nad_c <=> h_c + nadh_c + pyr_c
bounds normal: (-1000.0, 1000.0)
bounds mutant: (0, 0)
------------------------------------------------------------
gene: b1241
uniprot: P0A9Q7
number of reactions involved: 3
* ALCD2x : etoh_c + nad_c <=> acald_c + h_c + nadh_c
bounds normal: (-1000.0, 1000

# Isobutyric strain knockout model

In [6]:
# copy model to create mutant
mutant_isobutyric = model.copy()

# create list of genes to KO
ko_list = pd.read_csv('data/df_isobutyric_knockouts.csv', index_col=0).dropna(subset = ['id_bigg'])
ko_list.iML1515_idx = ko_list.iML1515_idx.astype(int)
#ko_list

Read LP format model from file /tmp/tmpwas1gbd8.lp
Reading time = 0.02 seconds
: 1877 rows, 5424 columns, 21150 nonzeros
Read LP format model from file /tmp/tmpi_u8yd5w.lp
Reading time = 0.02 seconds
: 1877 rows, 5424 columns, 21150 nonzeros


In [7]:
for i in ko_list.iML1515_idx:
    mutant_isobutyric.genes[i].knock_out()

    
#model = model
#mutant = mutant_isobutyric
#ko_list = ko_list.iML1515_idx

evaluate_knockout(model, mutant_isobutyric, ko_list.iML1515_idx)
    

gene: b0903
uniprot: P09373
number of reactions involved: 2
* PFL : coa_c + pyr_c --> accoa_c + for_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0, 0)
* OBTFL : 2obut_c + coa_c --> for_c + ppcoa_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0, 0)
------------------------------------------------------------
gene: b0871
uniprot: P07003
number of reactions involved: 1
* POX : h2o_c + pyr_c + q8_c --> ac_c + co2_c + q8h2_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0, 0)
------------------------------------------------------------
gene: b3114
uniprot: P42632
number of reactions involved: 2
* PFL : coa_c + pyr_c --> accoa_c + for_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0, 0)
* OBTFL : 2obut_c + coa_c --> for_c + ppcoa_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0, 0)
------------------------------------------------------------
gene: b3952
uniprot: P32675
number of reactions involved: 1
* PFL : coa_c + pyr_c --> accoa_c + for_c
bounds normal: (0.0, 1000.0)
bounds mutant: (0, 0

# Add Reactions
## Add Metabolites

In [8]:
# add metabolites
isobuta_c = Metabolite(
    'isobuta_c',
    formula='C4H8O2',
    name='2-methylpropanoic-acid',
    compartment='c')

isobuta_e = Metabolite(
    'isobuta_e',
    formula='C4H8O2',
    name='2-methylpropanoic-acid',
    compartment='e')

mppal_c = Metabolite(
    '2mppal_c',
    formula='C4H8O',
    name='2-methylpropanal',
    compartment='c')

h_c = model.metabolites.h_c
co2_c = model.metabolites.co2_c
mob_c = model.metabolites.get_by_id('3mob_c')
h2o_c = model.metabolites.h2o_c
nad_c = model.metabolites.nad_c
nadh_c = model.metabolites.nadh_c

## Isobutyric Mutant
### _Reaction 1: 3-Methyl-2-oxobutanoate => 2-methylpropanal_

In [9]:
# define reaction
reaction = Reaction('kivD_heterologous')
reaction.name = 'alpha-ketoisovalerate-decarboxylase'
reaction.subsystem = 'Alternate Carbon Metabolism'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [10]:
reaction.add_metabolites({
    mob_c: -1.0,
    h_c: -1.0,
    co2_c: 1.0,
    mppal_c: 1.0
})

reaction.reaction  

'3mob_c + h_c --> 2mppal_c + co2_c'

In [11]:
reaction.gene_reaction_rule = '( kivD )'
reaction.genes

frozenset({<Gene kivD at 0x7f2028274f10>})

In [12]:
mutant_isobutyric.add_reactions([reaction])

### _Reaction 2: 2-methylpropanal =>  2-methylpropanoic-acid_

In [13]:
# define reaction
reaction = Reaction('ALDD19xr_mppal')
reaction.name = 'Aldehyde dehydrogenase (2-methylpropanal, NAD)'
reaction.subsystem = 'Alternate Carbon Metabolism'
reaction.lower_bound = -1000.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [14]:
reaction.add_metabolites({
    h2o_c: -1.0,
    nad_c: -1.0, 
    mppal_c: -1.0,
    h_c: 2.0,
    nadh_c: 1.0,
    isobuta_c: 1.0
})

reaction.reaction  

'2mppal_c + h2o_c + nad_c <=> 2.0 h_c + isobuta_c + nadh_c'

In [15]:
reaction.gene_reaction_rule = '( b1385 )'
reaction.genes

frozenset({<Gene b1385 at 0x7f2060bb5890>})

In [16]:
mutant_isobutyric.add_reactions([reaction])

### _Reaction 3: 2-methylpropanoic-acid => exchange_

In [17]:
# define reaction
reaction = Reaction('Ex_isobuta')
reaction.name = '2-methylpropanoic-acid exchange'
reaction.subsystem = 'Extracellular exchange'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [18]:
reaction.add_metabolites({
    isobuta_c: -1.0,
    isobuta_e: 1.0
})

reaction.reaction  

'isobuta_c --> isobuta_e'

In [19]:
reaction.gene_reaction_rule = '( isobuta_transporter )'
reaction.genes

frozenset({<Gene isobuta_transporter at 0x7f2027161190>})

In [20]:
mutant_isobutyric.add_reactions([reaction])

## Write Model

In [24]:
cobra.io.write_sbml_model(mutant_isobutyric, "result/iML1515_isobutyric_strain.xml")